In [ ]:
import os
import tempfile

import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scvi
import seaborn as sns
import torch
from celldreamer.paths import DATA_DIR

from pathlib import Path

In [ ]:
raw_dataset_path = DATA_DIR / "raw" / "tabula_muris.h5ad"

adata = sc.read(
    raw_dataset_path,
    backup_url="https://figshare.com/ndownloader/files/23938934",
)

In [ ]:
adata.layers["X_counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(
    adata,
    flavor="seurat_v3",
    n_top_genes=2000,
    layer="X_counts",
    subset=True,
)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="tissue")

In [19]:
# sc.AnnData.write_h5ad(adata, DATA_DIR / "processed" / "tabula_muris.h5ad")

In [5]:
adata = sc.read_h5ad(DATA_DIR / "processed" / "tabula_muris.h5ad")

In [6]:
# adata.layers["X_counts"].A.max(1)